In [ ]:
import os
import win32com.client
import numpy as np
import re

# def replace_E(s, new_number):
#     # Reverse the string to process from the back
#     reversed_s = s[::-1]
#     # Find the positions of the operators in the reversed string
#     operator_positions = [m.start() for m in re.finditer(r'[\+\-]', reversed_s)]
#     # We need the second operator from the end, which is the second from the start in the reversed string
#     if len(operator_positions) < 2:
#         return s  # If there are less than 2 operators, return the original string
#     # The position of the second operator from the end in the original string
#     second_operator_pos = operator_positions[1]
#     # Reverse the position to get the correct index in the original string
#     index_in_original = len(s) - second_operator_pos - 1
#     # Find the end position of the number following the second operator
#     next_operator_match = re.search(r'[\+\-]', s[index_in_original+1:])
#     if next_operator_match:
#         end_pos = index_in_original + next_operator_match.start() + 1
#     else:
#         end_pos = len(s)
#     # Replace the second-to-last element including its sign
#     new_string = s[:index_in_original] + str(new_number) + s[end_pos:]
#     return new_string

def replace_E(s, new_number):
    # Find the position of the last operator in the string
    last_operator_match = re.search(r'[\+\-](?=[^+\-]*$)', s)
    last_operator_pos = last_operator_match.start()
    # Find the end position of the number following the last operator
    next_operator_match = re.search(r'[\+\-]', s[last_operator_pos+1:])
    if next_operator_match:
        end_pos = last_operator_pos + next_operator_match.start() + 1
    else:
        end_pos = len(s)
    # Replace the last parameter including its sign
    new_string = s[:last_operator_pos] + str(new_number) + s[end_pos:]
    return new_string

#define groups through which to access all utility functions
VHG_EinP = ['EinpA_bF','EinpA_kF','EinpA_nC','EinpA_nE','EinpF_bF','EinpF_kF','EinpF_nC','EinpF_nE','EinpK_bF','EinpK_kF','EinpK_nC','EinpK_nE','EinpU_bF','EinpU_kF','EinpU_nC','EinpU_nE']
Mode = ['cbike','ebike']

#Parameter values for E to try out (see Excel 003)
E = [['+0.2'],
    ['+0.3'],
    ['+0'],
    ['+0.3'],
    ['-0.8'],
    ['+0.3'],
    ['+0.5'],
    ['+0.4'],
    ['+0.8'],
    ['+0.1']]
# E[3][0]: 4th row (cbike GJSUUx), 1st column (0th iteration)

# Matrix numbers for calculating resulting modal share per trip purpose
mAW_cbike = 5306
mAx_cbike = 5221
mF_cbike = 5305
mFx_cbike = 5222
mBEK_cbike = 5308
mKx_cbike = 5223
mGJSU_cbike = 5310
mUx_cbike = 5224
mHY_cbike = 5312

mAW_ebike = 5307
mAx_ebike = 5225
mF_ebike = 5314
mFx_ebike = 5226
mBEK_ebike = 5309
mKx_ebike = 5227
mGJSU_ebike = 5311
mUx_ebike = 5228
mHY_ebike = 5313

mAW_all = 122
mAx_all = 20
mF_all = 126
mFx_all = 21
mBEK_all = 125
mKx_all = 22
mGJSU_all = 128
mUx_all = 23
mHY_all = 132

In [ ]:
#Loop over iteration steps to change E, rerun model, compute mode shares by trip purpose, and save
for i in range(1,3):
    #open ver
    #verPath = r'D:\ver\20240709_1200.ver'
    verPath = r'C:\Users\arning.FUL\Desktop\ver\DIFF_20250128_1135_E14_0.ver'
    Visum = win32com.client.Dispatch('Visum.Visum.24')
    Visum.loadVersion(verPath)
    
    #change utils for Einpendler visem
    visem_einp = Visum.Procedures.Operations.ItemByKey(46)
    for mode in Mode:
        for vhg in VHG_EinP:
            
            #get old utilityfunction
            util = visem_einp.VISEMCombinedParameters.VISEMModeChoiceParameters.VISEMDGroupModeChoiceParameters(vhg).VISEMDGroupModeModeChoiceParameters(mode).AttValue("UtilityFunction")
            
            #if utility isnt "-99999"... (which would mean that the respective bicycle isnt available for this person group)
            if util != "-99999":

                # Set E based on trip purpose
                if vhg == 'EinpA_bF' or vhg == 'EinpA_kF' or vhg == 'EinpA_nC' or vhg == 'EinpA_nE':
                    if mode == 'cbike':
                        new_E = E[0][i]
                    elif mode == 'ebike':
                        new_E = E[5][i]
                elif vhg == 'EinpK_bF' or vhg == 'EinpK_kF' or vhg == 'EinpK_nC' or vhg == 'EinpK_nE':
                    if mode == 'cbike':
                        new_E = E[2][i]
                    elif mode == 'ebike':
                        new_E = E[7][i]
                elif vhg == 'EinpF_bF' or vhg == 'EinpF_kF' or vhg == 'EinpF_nC' or vhg == 'EinpF_nE':
                    if mode == 'cbike':
                        new_E = E[1][i]
                    elif mode == 'ebike':
                        new_E = E[6][i]
                elif vhg == 'EinpU_bF' or vhg == 'EinpU_kF' or vhg == 'EinpU_nC' or vhg == 'EinpU_nE':
                    if mode == 'cbike':
                        new_E = E[3][i]
                    elif mode == 'ebike':
                        new_E = E[8][i]
                
                #write new util function into Visum
                new_util = replace_E(util,new_E)
                visem_einp.VISEMCombinedParameters.VISEMModeChoiceParameters.VISEMDGroupModeChoiceParameters(vhg).VISEMDGroupModeModeChoiceParameters(mode).SetAttValue("UtilityFunction", new_util)
                
    #change utils for main visem
    visem_main = Visum.Procedures.Operations.ItemByKey(47)
    VHG_main = [
    "BSch_bF", "BSch_kF", "BSch_nC", "BSch_nE",
    "ERWmP_bF", "ERWmP_kF", "ERWmP_nC", "ERWmP_nE",
    "ERWoP_bF", "ERWoP_kF", "ERWoP_nC", "ERWoP_nE",
    "GSch_bF", "GSch_kF", "GSch_nC", "GSch_nE",
    "Kind_bF", "Kind_kF", "Kind_nC", "Kind_nE",
    "NERWmP_bF", "NERWmP_kF", "NERWmP_nC", "NERWmP_nE",
    "NERWoP_bF", "NERWoP_kF", "NERWoP_nC", "NERWoP_nE",
    "Rentner_bF", "Rentner_kF", "Rentner_nC", "Rentner_nE",
    "Sch_bF", "Sch_kF", "Sch_nC", "Sch_nE",
    "Stud_bF", "Stud_kF", "Stud_nC", "Stud_nE"
    ]
    Activity = ["A", "Ax", "B", "E", "F", "Fx", "G", "H", "J", "K", "Kx","S", "U", "Ux", "W", "Y"]
    
    for mode in Mode:
        for vhg in VHG_main:
            for activity in Activity:
                #get old utility function
                #not all combinations of vhg and activities exist; skip those exception
                try:
                    util = visem_main.VISEMCombinedParameters.VISEMModeChoiceParameters.VISEMDGroupModeChoiceParameters(vhg).VISEMDGroupActivityModeModeChoiceParameters(activity,mode).AttValue("UtilityFunction")
                    if util != "-99999":
                        if mode == 'cbike':
                            if activity == 'A' or activity == 'Ax' or activity == 'W':
                                new_E = E[0][i]
                            elif activity == 'F' or activity == 'Fx':
                                new_E = E[1][i]
                            elif activity == 'B' or activity == 'E' or activity == 'K' or activity == 'Kx':
                                new_E = E[2][i]
                            elif activity == 'G' or activity == 'J' or activity == 'S' or activity == 'U' or activity == 'Ux':
                                new_E = E[3][i]
                            elif activity == 'H' or activity == 'Y':
                                new_E = E[4][i]
                        elif mode == "ebike":
                            if activity == 'A' or activity == 'Ax' or activity == 'W':
                                new_E = E[5][i]
                            elif activity == 'F' or activity == 'Fx':
                                new_E = E[6][i]
                            elif activity == 'B' or activity == 'E' or activity == 'K' or activity == 'Kx':
                                new_E = E[7][i]
                            elif activity == 'G' or activity == 'J' or activity == 'S' or activity == 'U' or activity == 'Ux':
                                new_E = E[8][i]
                            elif activity == 'H' or activity == 'Y':
                                new_E = E[9][i]                        

                        #write new utilityfunction into visum
                        new_util = replace_E(util,new_E)
                        visem_main.VISEMCombinedParameters.VISEMModeChoiceParameters.VISEMDGroupModeChoiceParameters(vhg).VISEMDGroupActivityModeModeChoiceParameters(activity,mode).SetAttValue("UtilityFunction", new_util)

                except Exception as e:
                    pass
                    # handle exception (skip the current iteration)
                    # print(f"Skipping vhg: {vhg}, activity: {activity}, mode: {mode} due to error: {e}")
    #run procedures
    Visum.Procedures.Execute()
    
    # Calculate mode share per persongroup
    print('iteration '+str(i)+'; cbike A, Ax, W:')
    print(round(((np.array(Visum.Net.Matrices.ItemByKey(mAW_cbike).GetValues()).sum() + np.array(Visum.Net.Matrices.ItemByKey(mAx_cbike).GetValues()).sum())/
                 (np.array(Visum.Net.Matrices.ItemByKey(mAW_all).GetValues()).sum() + np.array(Visum.Net.Matrices.ItemByKey(mAx_all).GetValues()).sum()))*100, 1)
         )
    print('iteration '+str(i)+'; cbike F, Fx:')
    print(round(((np.array(Visum.Net.Matrices.ItemByKey(mF_cbike).GetValues()).sum() + np.array(Visum.Net.Matrices.ItemByKey(mFx_cbike).GetValues()).sum())/
                 (np.array(Visum.Net.Matrices.ItemByKey(mF_all).GetValues()).sum() + np.array(Visum.Net.Matrices.ItemByKey(mFx_all).GetValues()).sum()))*100, 1)
         )
    print('iteration '+str(i)+'; cbike B, E, K, Kx:')
    print(round(((np.array(Visum.Net.Matrices.ItemByKey(mBEK_cbike).GetValues()).sum() + np.array(Visum.Net.Matrices.ItemByKey(mKx_cbike).GetValues()).sum())/
                 (np.array(Visum.Net.Matrices.ItemByKey(mBEK_all).GetValues()).sum() + np.array(Visum.Net.Matrices.ItemByKey(mKx_all).GetValues()).sum()))*100, 1)
         )
    print('iteration '+str(i)+'; cbike G, J, S, U, Ux:')
    print(round(((np.array(Visum.Net.Matrices.ItemByKey(mGJSU_cbike).GetValues()).sum() + np.array(Visum.Net.Matrices.ItemByKey(mUx_cbike).GetValues()).sum())/
                 (np.array(Visum.Net.Matrices.ItemByKey(mGJSU_all).GetValues()).sum() + np.array(Visum.Net.Matrices.ItemByKey(mUx_all).GetValues()).sum()))*100, 1)
         )
    print('iteration '+str(i)+'; cbike H, Y:')
    print(round(((np.array(Visum.Net.Matrices.ItemByKey(mHY_cbike).GetValues()).sum()) /
                 (np.array(Visum.Net.Matrices.ItemByKey(mHY_all).GetValues()).sum())) * 100, 1)
         )

    print('iteration '+str(i)+'; ebike A, Ax, W:')
    print(round(((np.array(Visum.Net.Matrices.ItemByKey(mAW_ebike).GetValues()).sum() + np.array(Visum.Net.Matrices.ItemByKey(mAx_ebike).GetValues()).sum())/
                 (np.array(Visum.Net.Matrices.ItemByKey(mAW_all).GetValues()).sum() + np.array(Visum.Net.Matrices.ItemByKey(mAx_all).GetValues()).sum()))*100, 1)
         )
    print('iteration '+str(i)+'; ebike F, Fx:')
    print(round(((np.array(Visum.Net.Matrices.ItemByKey(mF_ebike).GetValues()).sum() + np.array(Visum.Net.Matrices.ItemByKey(mFx_ebike).GetValues()).sum())/
                 (np.array(Visum.Net.Matrices.ItemByKey(mF_all).GetValues()).sum() + np.array(Visum.Net.Matrices.ItemByKey(mFx_all).GetValues()).sum()))*100, 1)
         )
    print('iteration '+str(i)+'; ebike B, E, K, Kx:')
    print(round(((np.array(Visum.Net.Matrices.ItemByKey(mBEK_ebike).GetValues()).sum() + np.array(Visum.Net.Matrices.ItemByKey(mKx_ebike).GetValues()).sum())/
                 (np.array(Visum.Net.Matrices.ItemByKey(mBEK_all).GetValues()).sum() + np.array(Visum.Net.Matrices.ItemByKey(mKx_all).GetValues()).sum()))*100, 1)
         )
    print('iteration '+str(i)+'; ebike G, J, S, U, Ux:')
    print(round(((np.array(Visum.Net.Matrices.ItemByKey(mGJSU_ebike).GetValues()).sum() + np.array(Visum.Net.Matrices.ItemByKey(mUx_ebike).GetValues()).sum())/
                 (np.array(Visum.Net.Matrices.ItemByKey(mGJSU_all).GetValues()).sum() + np.array(Visum.Net.Matrices.ItemByKey(mUx_all).GetValues()).sum()))*100, 1)
         )
    print('iteration '+str(i)+'; ebike H, Y:')
    print(round(((np.array(Visum.Net.Matrices.ItemByKey(mHY_ebike).GetValues()).sum())/
                 (np.array(Visum.Net.Matrices.ItemByKey(mHY_all).GetValues()).sum()))*100, 1)
         )
    #save
    #verPath_append = r'D:\ver\20240709_1200_'+str(i)+'.ver'
    verPath_append = r'C:\Users\arning.FUL\Desktop\ver\DIFF_20250128_1135_E14_'+str(i)+'.ver'
    Visum.saveVersion(verPath_append)
    Visum = None